# ELT Sample: Azure Blob Stroage - Databricks - SQLDW
In this notebook, you extract data from Azure Blob Storage into Databricks cluster, run transformations on the data in Databricks cluster, and then load the transformed data into Azure SQL Data Warehouse.

## prerequisites:
- Azure Blob Storage Account and Containers
- Databricks Cluster (Spark)
- Azure SQL Data Warehouse

## Sample data
- https://github.com/Azure/usql/blob/master/Examples/Samples/Data/json/radiowebsite/small_radio_json.json

## LINKS
- https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-storage.html
- https://docs.azuredatabricks.net/spark/latest/data-sources/azure/sql-data-warehouse.html
- [Quickstart: Create an Azure SQL Data Warehouse](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-portal)

# Connecting to Azure Blob Storage and access a sample Json file

## Set up an account access key

In [4]:
# Set up an account access key
# spark.conf.set(
#  "fs.azure.account.key.<storage-account-name>.blob.core.windows.net",
#  "<storage-access-key>")

spark.conf.set(
  "fs.azure.account.key.databrickstore.blob.core.windows.net",
  "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg==")

Once an account access key or a SAS is set up in your notebook, you can use standard Spark and Databricks APIs to read from the storage account

In [6]:
# dbutils.fs.ls("wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>")
dbutils.fs.ls("wasbs://dbdemo01@databrickstore.blob.core.windows.net")

## Mount a Blob storage container or a folder inside a container

In [8]:
# mount a Blob storage container or a folder inside a container
# dbutils.fs.mount(
#   source = "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>",
#   mount_point = "<mount-point-path>",
#   extra_configs = <"<conf-key>": "<conf-value>">)
# [note] <mount_point> is a DBFS path and the path must be under /mnt

dbutils.fs.mount(
  source = "wasbs://dbdemo01@databrickstore.blob.core.windows.net",
  mount_point = "/mnt/dbdemo01",
  extra_configs = {"fs.azure.account.key.databrickstore.blob.core.windows.net": "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg=="})

## Access files in your container as if they were local files

In [11]:
# Access files in your container as if they were local files
# (TEXT) df = spark.read.text("/mnt/%s/...." % <mount-point-path>)
# (JSON) df = spark.read.json("/mnt/%s/...." % <mount-point-path>)

df = spark.read.json( "/mnt/%s/small_radio_json.json" % "dbdemo01" )

# display(df)
df.show()

## Unmount the blob storage (if needed)

In [13]:
# unmount
# dbutils.fs.unmount("<mount-point-path>")
# dbutils.fs.unmount("/mnt/dbdemo01")

# Transform data in Azure Databricks

Start by retrieving only the columns firstName, lastName, gender, location, and level from the dataframe you already created.

In [16]:
specificColumnsDf = df.select("firstname", "lastname", "gender", "location", "level")
specificColumnsDf.show()

You can further transform this data to rename the column level to subscription_type.

In [18]:
renamedColumnsDF = specificColumnsDf.withColumnRenamed("level", "subscription_type")
renamedColumnsDF.show()

# Load data into Azure SQL Data Warehouse

In [20]:
# Apply some transformations to the data, then use the
# Data Source API to write the data back to another table in SQL DW.

# [note] the SQL date warehouse connector uses Azure Blob Storage as a temporary storage to upload data between Azure Databricks and Azure SQL Data Warehouse.

## SQL Data Warehouse related settings
dwTable= "mytable001"
dwDatabase = "sqldwdemo001"
dwServer = "sqldwdemoserver001" 
dwUser = "yoichika"
dwPass = "P@ssw0rd____"
dwJdbcPort =  "1433"
dwJdbcExtraOptions = "encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
sqlDwUrl = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwPass + ";$dwJdbcExtraOptions"
sqlDwUrlSmall = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwPass


tempDir = "wasbs://dbdemo01tmp@databrickstore.blob.core.windows.net/tempDirs"

#sc._jsc.hadoopConfiguration().set(
#  "fs.azure.account.key.<your-storage-account-name>.blob.core.windows.net",
#  "<your-storage-account-access-key>")
acntInfo = "fs.azure.account.key.databrickstore.blob.core.windows.net"
sc._jsc.hadoopConfiguration().set(
  acntInfo, 
  "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg==")

## Loading transformed dataframe (renamedColumnsDF) into SQLDW
spark.conf.set("spark.sql.parquet.writeLegacyFormat","true")

## This snippet creates a table called 'dwTable' in the SQL database.
#df.write \
#  .format("com.databricks.spark.sqldw") \
#  .option("url", "jdbc:sqlserver://<the-rest-of-the-connection-string>") \
#  .option("forward_spark_azure_storage_credentials", "true") \
#  .option("dbtable", "my_table_in_dw_copy") \
#  .option("tempdir", "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>") \
#  .save()

renamedColumnsDF.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", sqlDwUrlSmall) \
  .option("dbtable", dwTable) \
  .option( "forward_spark_azure_storage_credentials","true") \
  .option("tempdir", tempDir) \
  .mode("overwrite") \
  .save()